### Fabio Valerio Massoli, Lucia Vadicamo, Fabrizio Falchi, Giuseppe Amato (ISTI-CNR)

Jupyter Notebook to create MarkDown files from Excel file that contains article details

In [9]:
import pandas as pd 
import numpy as np
from mdutils.mdutils import MdUtils
from mdutils import Html
from IPython.display import Markdown, display

title="A Leap among Quantum Computing and Quantum Neural Networks: A Survey"
authors="Fabio Valerio Massoli, Lucia Vadicamo, Fabrizio Falchi, Giuseppe Amato"
our_paper_url="https://arxiv.org/abs/2107.03313"
k=2 #minimum occurrece of a topic used to create the 'papersSortedByTopics' MarkDown file

# Loading  "LeapIntoQuantumComputationsPaperList.xlsx" data

In [10]:
df = pd.read_excel("LeapIntoQuantumComputationsPaperList.xlsx",sheet_name='all', usecols= 'A:N',engine='openpyxl') 
df = df.dropna(subset=['Title'])
df=df[['bibTex','URL','year','Author','Title','NameJournalConf','EntryType','Topic','Model']]
df.fillna('na', inplace=True)
surveys=df[df["EntryType"]=="survey"]
papers= df[df["EntryType"]!="survey"]
df['year']=pd.to_numeric(df['year'], downcast='signed')
print("*********************************************************")
print("**DataFrame details **")
print(df.info())
print("*********************************************************")
print(f"**last document in the list:\n{df.iloc[-1]}**")


#TOPIC LIST
allTopic=[t.strip() for  tt in df["Topic"]for t in tt.split(";") if tt!='na' and t!=' ' and t!='']
topicList_with_occurrences =list(zip(*np.unique(allTopic, return_counts=True))) #
print("*********************************************************")
print("**TOPICS with occurrences**")
for t in topicList_with_occurrences:
    print (t)
topk_topics=np.sort([t for t,count in topicList_with_occurrences if count>k])
print("*********************************************************")
print(f"**TOPICS with occurrence>{k} (total={len(topk_topics)}) **")
print(topk_topics)

*********************************************************
**DataFrame details **
<class 'pandas.core.frame.DataFrame'>
Int64Index: 184 entries, 0 to 183
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   bibTex           184 non-null    object
 1   URL              184 non-null    object
 2   year             184 non-null    int16 
 3   Author           184 non-null    object
 4   Title            184 non-null    object
 5   NameJournalConf  184 non-null    object
 6   EntryType        184 non-null    object
 7   Topic            184 non-null    object
 8   Model            184 non-null    object
dtypes: int16(1), object(8)
memory usage: 13.3+ KB
None
*********************************************************
**last document in the list:
bibTex             @article{zoufal2019quantum,\n  title={Quantum ...
URL                https://www.nature.com/articles/s41534-019-0223-2
year                                   

# Creating MD files



In [17]:
def write_item (row,mdFile):
    topics= " ".join(['__`['+ s.strip() +']`__' for s in row['Topic'].split(';') if s!='na'] )
    model=" ".join(['_`('+ s.strip() +')`_' for s in row['Model'].split(';') if s!='na'])
    #topics= " ".join(['<code style="background-color: Gray; color: white">['+ s.strip() +']</code>' for s in row['Topic'].split(';') if s!='na'] )
    #model=" ".join(['<code style="background-color: DarkCyan; color: white">['+ s.strip() +']</code>' for s in row['Model'].split(';') if s!='na'])
    newentry=f"- [{row['Author']}] {mdFile.new_inline_link(link=row['URL'] , text='**'+row['Title'].strip()+'.**')} _{row['NameJournalConf']}_. {topics} {model}" 
    bib=f"<details><summary>Bibtex</summary><pre><code>{row['bibTex']}</code></pre></details>\n "
    mdFile.write(newentry+bib)

def write_item2 (row,file):
    prefix="<details><summary>"
    topics= " ".join(['<code><b>['+ s.strip() +']</b></code>' for s in row['Topic'].split(';') if s!='na'] )
    model=" ".join(['<code><i>('+ s.strip() +')</i></code>' for s in row['Model'].split(';') if s!='na'])
    newentry=f"[{row['Author']}] <b><a href={row['URL']}> {row['Title'].strip()}. </a></b> <i>{row['NameJournalConf']}.</i> {topics} {model}" 
    bib=f"</summary><pre><code>{row['bibTex']}</code></pre>"
    suffix="</details>\n"
    file.write(prefix+newentry+bib+suffix)
    
def writeTOC (values, file, sep='\n'):
    for y in values:
        ss="-".join(str(y).split())
        file.write(f"* [{str(y)}](#{ss}){sep}")

def writePaperByTopics (topics, data, file):
    for t in topics:
        file.new_paragraph(f"# {str(t)}\n")
        selIndex=[i  for i in data.index if t in data.loc[i]["Topic"]]        
        sel=data.loc[selIndex]
        years=np.sort(sel["year"].astype('int32').unique())[::-1]
        for y in years:
            file.new_paragraph(f"### {str(y)}\n")
            sy=sel[sel["year"]==y]
            for index, row in sy.iterrows():
                write_item2(row,file)

def writePaperByYear (years, data, file):
    for y in years:
        file.new_paragraph(f"### {str(y)}\n")
        sy=data[data["year"]==y]
        for index, row in sy.iterrows():
            write_item2(row,file)

form_link="https://forms.gle/75AdK8yPL1N4hJ9H7"    
################### README ############################
mdFile = MdUtils(file_name='README', title=title)

mdFile.new_paragraph("This repo reports a collection of relevant papers in the fields of Quantum Computing, Quantum Machine Learning and Quantum Deep Learning.")
mdFile.new_paragraph()

mdFile.new_paragraph(f"These articles represented the starting point for the paper "+ mdFile.new_inline_link(link=our_paper_url, text=title) +" and for anyone who is approaching such a field. ")
mdFile.write("We are continuously improving the project. Welcome to Pull Request the works (papers, repositories, topics) that are missed by the repo")

mdFile.new_paragraph('To cite our work, please use the following reference')
mdFile.new_paragraph("<pre><code>@article{massoli2021leap,\n title = {"+title+"},\n author = {"+authors+"},\n year={2021},\n eprint={2107.03313},\n archivePrefix={arXiv},\n primaryClass={quant-ph}\n }</code></pre>")

mdFile.new_paragraph("# Be part of the team!")
mdFile.new_paragraph("Have you noticed a missing or misclassified article? We would greatly appreciate it if you could report it to us using the form at the following link: " + mdFile.new_inline_link(link=form_link, text=form_link))
mdFile.new_paragraph('')


mdFile.new_paragraph("## Table of Contents")
mdFile.new_paragraph("* [All the papers (flat view)](https://github.com/fvmassoli/survey-quantum-computations/blob/master/allPapers.md)")
mdFile.new_paragraph("* [Papers divided by topics](https://github.com/fvmassoli/survey-quantum-computations/blob/master/papersSortedByTopics.md) [WIP]")
mdFile.new_paragraph("## Contacts") 
mdFile.new_paragraph('If you have any question about our work, please contact [Dr. Fabio Valerio Massoli](mailto:fabio.massoli@isti.cnr.it) and [Dr. Lucia Vadicamo](mailto:lucia.vadicamo@isti.cnr.it)\n')
mdFile.new_paragraph('Have fun! :-D')

mdFile.create_md_file()
###############################################


################### FLATVIEW ############################
mdFile = MdUtils(file_name='allPapers', title=title)
mdFile.new_paragraph("# Table of Contents\n")
mdFile.new_paragraph("* [Surveys](#Surveys)\n")
yearsSurvey=np.sort(surveys["year"].astype('int32').unique())[::-1]
mdFile.write("\t")
writeTOC(yearsSurvey,mdFile,'\t')
mdFile.new_paragraph("* [Papers](#Papers)\n")
years=np.sort(papers["year"].astype('int32').unique())[::-1]
mdFile.write("\t")
writeTOC(years,mdFile, '\t')
mdFile.new_paragraph()

#surveys
mdFile.new_paragraph("# Surveys")#mdFile.new_header(level=1, title="Surveys")
mdFile.new_paragraph()

writePaperByYear (yearsSurvey, surveys, mdFile)

#papers
mdFile.new_paragraph("# Papers")#mdFile.new_header(level=2, title="Papers")
mdFile.new_paragraph()

writePaperByYear (years, papers, mdFile)

mdFile.create_md_file()
###############################################

################### papersSortedByTopics ############################
#only topics with more than 2 papers are used
mdFile = MdUtils(file_name='papersSortedByTopics', title=title)
mdFile.new_paragraph('--Work in Progress--')
mdFile.new_paragraph("# Table of Contents\n")
writeTOC(topk_topics,mdFile)
mdFile.write("\n")
writePaperByTopics(topk_topics, df, mdFile)
mdFile.create_md_file()


In [18]:
display(Markdown("README.md"))


A Leap among Quantum Computing and Quantum Neural Networks: A Survey
====================================================================


This repo reports a collection of relevant papers in the fields of Quantum Computing, Quantum Machine Learning and Quantum Deep Learning.



These articles represented the starting point for the paper [A Leap among Quantum Computing and Quantum Neural Networks: A Survey](https://arxiv.org/abs/2107.03313) and for anyone who is approaching such a field. We are continuously improving the project. Welcome to Pull Request the works (papers, repositories, topics) that are missed by the repo

To cite our work, please use the following reference

<pre><code>@article{massoli2021leap,
 title = {A Leap among Quantum Computing and Quantum Neural Networks: A Survey},
 author = {Fabio Valerio Massoli, Lucia Vadicamo, Fabrizio Falchi, Giuseppe Amato},
 year={2021},
 eprint={2107.03313},
 archivePrefix={arXiv},
 primaryClass={quant-ph}
 }</code></pre>

# Be part of the team!

Have you noticed a missing or misclassified article? We would greatly appreciate it if you could report it to us using the form at the following link: [https://forms.gle/75AdK8yPL1N4hJ9H7](https://forms.gle/75AdK8yPL1N4hJ9H7)



## Table of Contents

* [All the papers (flat view)](https://github.com/fvmassoli/survey-quantum-computations/blob/master/allPapers.md)

* [Papers divided by topics](https://github.com/fvmassoli/survey-quantum-computations/blob/master/papersSortedByTopics.md) [WIP]

## Contacts

If you have any question about our work, please contact [Dr. Fabio Valerio Massoli](mailto:fabio.massoli@isti.cnr.it) and [Dr. Lucia Vadicamo](mailto:lucia.vadicamo@isti.cnr.it)


Have fun! :-D

In [13]:
#display(Markdown(open('allPapers.md', encoding='utf-8').read()))

In [14]:
#display(Markdown(open('papersSortedByTopics.md', encoding='utf-8').read()))